In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as sciconsts
import scipy.fftpack as fft
import scipy.interpolate as sci_interpolate
import scipy.integrate as sci_integrate
import scipy.signal as signal

In [3]:
''' caozhoujian function taylorT2'''
def chirp_mass(m1,m2):
    return np.power(m1*m2,0.6)/(np.power(m1+m2,0.2))
def sym_mass_ratio(m1,m2):
    return m1*m2/(np.power(m1+m2,2.0))
def F_cross(theta,psi,phi):
    return 0.5*(1+np.power(np.cos(theta),2))*np.cos(2*phi)*np.sin(2*psi)-np.cos(theta)*np.sin(2*phi)*np.cos(2*psi)
def F_plus(theta,psi,phi):
    return -0.5*(1+np.power(np.cos(theta),2))*np.cos(2.0*phi)*np.cos(2*psi)-np.cos(theta)*np.sin(2*phi)*np.sin(2*psi)
def D_eff(D,theta,psi,phi,iota):
    return D/(np.sqrt(np.power(F_cross(theta,psi,phi),2.0)*np.power(np.cos(iota),2.0)+np.power(F_plus(theta,psi,phi),2)*0.25*(1+np.power(np.cos(iota),2))))
def Psi(f,m_c,smr,D,theta,psi,phi,iota,t_c,phi_c,v_lso=1/np.sqrt(6.0)):
    Mcal=m_c/np.power(smr,0.6)
    vcal=np.power(sciconsts.pi*M*f,1/3.0)
    return 2*sciconsts.pi*f*t_c-phi_c-sciconsts.pi*0.25+3.0/(128.0*smr*np.power(v,5.0))*(1+20/9.0*(743/336.0+11/4.0*smr)*np.power(v,2.0)-16.0*sciconsts.pi*np.power(v,3.0)+10.0*(3058673.0/1016064.0+5429/1008.0*smr+617.0/144.0*np.power(smr,2.0))*np.power(v,4.0)+sciconsts.pi*(38645.0/756.0-65/9.0*smr)*(1+3*np.log(v/v_lso))*np.power(v,5.0)+(11583231236531.0/4694215680.0+640.0/3.0*np.power(sciconsts.pi,2)-6848.0/21.0*0.57721566490153286060651209-6848/21.0*np.log(4.0*v)+(-15737765635.0/3048192+2255.0*np.power(sciconsts.pi,2.0)/12.0)*smr+76055.0/1728*np.power(smr,2.0)-127825/1296*np.power(smr,3.0))*np.power(v,6.0)+sciconsts.pi*(77096675.0/254016.0+378515.0/1512.0*smr-74045.0/756.0*np.power(smr,2.0))*np.power(v,7.0))
def h(f,m_c,smr,D,theta,psi,phi,iota,t_c,phi_c):
    return np.power(m_c,5.0/6.0)/D_eff(D,theta,psi,phi,iota)*np.power(f,-7.0/6.0)*np.exp(j*Psi(f,m_c,smr,D,theta,psi,phi,iota,t_c,phi_c))

'''MY FUNCTION PROJECT1'''
def k_1cal(m_1,m_2):
    return 64.0*m_1*m_2*(m_1+m_2)/5.0
def t_0cal(m_1,m_2,t_gas,a_0):
    return t_gas*0.25*np.log(k_1cal(m_1,m_2)+np.power(a_0,4.0)/t_gas)
def acal(t,k_1,t_gas,t_0):
    return np.power((np.exp(4.0*(t_0-t)/t_gas)-k_1)*t_gas,0.25)

def D1acal(t,k_1,t_gas):
    a1=acal(t,k_1,t_gas,t_0)
    return -k_1*np.power(a1,-3.0)-a1/t_gas
def D2acal(t,k_1,t_gas):
    a2=acal(t,k_1,t_gas,t_0)
    D1a2=acal(t,k_1,t_gas,t_0)
    return (3.0*k_1/np.power(a2,4.0)+1/t_gas)*D1a2
def Omegacal(t,k_1,t_gas,t_0,m_1,m_2):
    return np.power((m_1+m_2)/np.power(acal(t,k_1,t_gas,t_0),3.0),0.5)
def D1Omegacal(t,k_1,t_gas,t_0,m_1,m_2):
    a3=acal(t,k_1,t_gas,t_0)
    D1a3=D1acal(t,k_1,t_gas)
    o1=Omegacal(t,k_1,t_gas,t_0,m_1,m_2)
    return -1.5*D1a3/a3*o1
def D2Omegacal(t,k_1,t_gas,t_0,m_1,m_2):
    a4=acal(t,k_1,t_gas,t_0)
    D1a4=D1acal(t,k_1,t_gas)
    D2a4=D2acal(t,k_1,t_gas)
    o2=Omegacal(t,k_1,t_gas,t_0,m_1,m_2)
    return 15.0/4.0*np.power(D1a4/a4,2.0)-1.5*D2a4/a4*o2
#reduced D^2(q_ij)
#D^2(q_xx-q_yy)    1
#D^2(q_xx+q_yy)    2
#D^2q_xy           3
#D^2q_zz=-D^2(q_xx+q_yy) 4
'''def helpnumber(n,t,r,k_1,t_gas,t_0,m_1,m_2):
    mu5=m_1*m_2/(m_1+m_2)
    a5=acal(t,k_1,t_gas,t_0)
    D1a5=D1Omegacal(t,k_1,t_gas,t_0,m_1,m_2)
    D2a5=D2Omegacal(t,k_1,t_gas,t_0,m_1,m_2)
    o5=Omegacal(t,k_1,t_gas,t_0,m_1,m_2)
    D1o5=D1Omegacal(t,k_1,t_gas,t_0,m_1,m_2)
    D2o5=D2Omegacal(t,k_1,t_gas,t_0,m_1,m_2)
    if n==1:
        return mu5*(-4.0*np.power(a5,2.0)*np.cos(2.0*t*o5)*np.power(o5,2.0)-8.0*a5*o5*np.sin(2.0*t*o5)*D1a5+2.0*np.cos(2.0*t*o5)*np.power(D1a5,2.0)-8.0*t*np.power(a5,2.0)*np.cos(2.0*t*o5)*o5*D1o5-4.0*np.power(a5,2.0)*np.sin(2.0*t*o5)*D1o5-8.0*t*a5*np.sin(2.0*t*o5)*D1o5*D1a5+4.0*np.power(t*a5,2.0)*np.cos(2.0*t*o5)*np.power(D1o5,2.0)+2.0*a5*np.cos(2.0*t*o5)*D2a5-2.0*t*np.power(a5,2.0)*np.sin(2.0*t*o5)*D2o5)
    elif n==3:
        return mu5*(-4.0*np.power(a5,2.0)*np.sin(2.0*t*o5)*np.power(o5,2.0)+8.0*a5*o5*np.cos(2.0*t*o5)*D1a5+2.0*np.sin(2.0*t*o5)*np.power(D1a5,2.0)-8.0*t*np.power(a5,2.0)*np.sin(2.0*t*o5)*o5*D1o5+4.0*np.power(a5,2.0)*np.cos(2.0*t*o5)*D1o5+8.0*t*a5*np.cos(2.0*t*o5)*D1o5*D1a5+4.0*np.power(t*a5,2.0)*np.sin(2.0*t*o5)*np.power(D1o5,2.0)+2.0*a5*np.sin(2.0*t*o5)*D2a5+2.0*t*np.power(a5,2.0)*np.cos(2.0*t*o5)*D2o5)
    elif n==2:
        return 2.0/3.0*mu5(D2a5*a5+np.power(D1a5,2.0))
    elif n==4:
        return -2.0/3.0*mu5(D2a5*a5+np.power(D1a5,2.0))''' 

'''all in SI unit'''
def h_plus(t,r,theta,k_1,t_gas,t_0,m_1,m_2):
    mu5=m_1*m_2/(m_1+m_2)
    a5=acal(t,k_1,t_gas,t_0)
    D1a5=D1Omegacal(t,k_1,t_gas,t_0,m_1,m_2)
    D2a5=D2Omegacal(t,k_1,t_gas,t_0,m_1,m_2)
    o5=Omegacal(t,k_1,t_gas,t_0,m_1,m_2)
    D1o5=D1Omegacal(t,k_1,t_gas,t_0,m_1,m_2)
    D2o5=D2Omegacal(t,k_1,t_gas,t_0,m_1,m_2)
    hp1_1=mu5*(-4.0*np.power(a5,2.0)*np.cos(2.0*t*o5)*np.power(o5,2.0)-8.0*a5*o5*np.sin(2.0*t*o5)*D1a5+2.0*np.cos(2.0*t*o5)*np.power(D1a5,2.0)-8.0*t*np.power(a5,2.0)*np.cos(2.0*t*o5)*o5*D1o5-4.0*np.power(a5,2.0)*np.sin(2.0*t*o5)*D1o5-8.0*t*a5*np.sin(2.0*t*o5)*D1o5*D1a5+4.0*np.power(t*a5,2.0)*np.cos(2.0*t*o5)*np.power(D1o5,2.0)+2.0*a5*np.cos(2.0*t*o5)*D2a5-2.0*t*np.power(a5,2.0)*np.sin(2.0*t*o5)*D2o5)
    hp2_3=2.0/3.0*mu5*(D2a5*a5+np.power(D1a5,2.0))
    return 1.0/r*(1.0/2.0*(1+np.power(np.cos(theta),2.0))*hp1_1-np.power(np.sin(theta),2.0)*1.5*hp2_3)
def h_cross(t,r,theta,k_1,t_gas,t_0,m_1,m_2):
    mu5=m_1*m_2/(m_1+m_2)
    a5=acal(t,k_1,t_gas,t_0)
    D1a5=D1Omegacal(t,k_1,t_gas,t_0,m_1,m_2)
    D2a5=D2Omegacal(t,k_1,t_gas,t_0,m_1,m_2)
    o5=Omegacal(t,k_1,t_gas,t_0,m_1,m_2)
    D1o5=D1Omegacal(t,k_1,t_gas,t_0,m_1,m_2)
    D2o5=D2Omegacal(t,k_1,t_gas,t_0,m_1,m_2)
    hp2_2=mu5*(-4.0*np.power(a5,2.0)*np.sin(2.0*t*o5)*np.power(o5,2.0)+8.0*a5*o5*np.cos(2.0*t*o5)*D1a5+2.0*np.sin(2.0*t*o5)*np.power(D1a5,2.0)-8.0*t*np.power(a5,2.0)*np.sin(2.0*t*o5)*o5*D1o5+4.0*np.power(a5,2.0)*np.cos(2.0*t*o5)*D1o5+8.0*t*a5*np.cos(2.0*t*o5)*D1o5*D1a5+4.0*np.power(t*a5,2.0)*np.sin(2.0*t*o5)*np.power(D1o5,2.0)+2.0*a5*np.sin(2.0*t*o5)*D2a5+2.0*t*np.power(a5,2.0)*np.cos(2.0*t*o5)*D2o5)
    return 1.0/r*np.cos(theta)*hp2_2

'''lisa noise curve in rpsd unit N2A5:laserpower 2w,40cm telescope,armlength 5e6 km'''

def S_gal_N2A5(f):
    if f>=1.0e-5 and f<1.0e-3:
        return np.power(f,-2.3)*np.power(10,-44.62)*20.0/3.0
    if f>=1.0e-3 and f<np.power(10,-2.7):
        return np.power(f,-4.4)*np.power(10,-50.92)*20.0/3.0
    if f>=np.power(10,-2.7) and f<np.power(10,-2.4):
        return np.power(f,-8.8)*np.power(10,-62.8)*20.0/3.0
    if f>=np.power(10,-2.4) and f<=0.01:
        return np.power(f,-20.0)*np.power(10,-89.68)*20.0/3.0
    if f>0.01 or f<1.0e-5:
        return 0
def S_n_lisa(f):
    m1=5.0e9
    m2=0.41*sciconsts.c/m1/2.0
    return (20.0/3.0*(1+np.power(f/m2,2.0))*(4.0*(9.0e-30/np.power(2*sciconsts.pi*f,4.0)*(1+1.0e-4/f))+2.96e-23+2.65e-23)/np.power(m1,2.0)+S_gal_N2A5(f))/sciconsts.c

In [6]:
m_sun=1.9891e30*sciconsts.G/np.power(sciconsts.c,3.0)
m_1=10*m_sun
m_2=10*m_sun
years=365.0*24*3600
t_gas=1000.0*years
a_0=3.0e8/sciconsts.c
m_sun

4.9270380555784728e-06

In [81]:
k_1=k_1cal(m_1,m_2)
t_0=t_0cal(m_1,m_2,t_gas,a_0)
theta=0.0
N=10000000
t_scale=4*years
r=10*1e6*3.261*sciconsts.light_year/sciconsts.c
t=np.linspace(0,t_scale,num=N)
(t_scale*0.01*2.56,N/0.01/t_scale,t_scale) 

(3229286.4, 7.927447995941146, 126144000.0)

In [82]:
T=t_scale/N
fs=1.0/T
1.0/t_scale

7.927447995941146e-09

In [83]:
y=np.zeros(N)
y=h_plus(t,r,theta,k_1,t_gas,t_0,m_1,m_2)

In [84]:
xs=np.linspace(0,1/2.0/T,num=N//2)
yff=fft.fft(y)
y_abs=abs(yff)
ys=np.zeros((N//2,), dtype=np.float64)
ys=2.0/N*y_abs[0:N//2]

In [85]:
np.sqrt(sci_integrate.quad(lambda x:4.0*h1(x)*h1(x)/S_n_lisa(x),1.0e-4,0.01))

array([  5.56174633e-10,   6.23672866e-10])

In [86]:
h1=sci_interpolate.interp1d(xs,ys,kind='cubic')
np.sqrt(sci_integrate.quad(lambda x:4.0*h1(x)*h1(x)/S_n_lisa(x),1.0e-8,0.01))

array([  1.13879518e-09,   1.48936725e-09])

In [53]:
fig1=plt.figure(figsize=(18,18),dpi=50)

In [54]:
plt.plot(xs,h1(xs),color="red",linestyle="-",label=r"1")
plt.xlabel(r'$f/Hz$',fontsize=18)
plt.ylabel(r'$rpsd$',fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.yscale('log')
plt.xscale('log')
plt.legend(loc='upper left', frameon=False,fontsize='xx-large')
plt.savefig('14.png')